Задание из видео Карпова [Разбор тестового задания на позицию junior аналитика данных](https://www.youtube.com/watch?v=3VWDh-gdiYg)

Описание данных:  
orders - таблица с информацией о заказах;  
order_id - уникальный идентификатор заказа;  
customer_id - уникальный идентификатор клиента;  
order_date - дата, когда заказ был создан;  
order_amount - cумма заказа;  
platform - платформа, через которую был сделан заказ (например, 'mobile', 'desktop');  
category - категория товара, который был заказан (например, 'smartphones', 'laptops').

1. Построй когортый анализ: выдели клиентов, совершивших первую покупку в течение определенного месяца (когортную дату) и вычисли размер каждой когорты.  
2. Рассчитай средний джоход с клиента для каждой когорты за 10 дней с момента их первой покупки.  
3. Отобрази результаты по месяцам первых покупок, категориям товаров и платформам.  

Вывод должен содержать следующие поля:
- дата когорты;
- платформа;
- категория товара;
- размер когорты;
- средний доход с клиента.

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [159]:
customer_ids = np.array([])
ids = []
for i in range(1,14):
    ids.extend(range(1, i*10 + 1))
    customer_ids = np.append(customer_ids, np.random.choice(ids, (1, i * 100 + 1)))

In [160]:
customer_ids = customer_ids.astype(int)

In [161]:
# число заказов
N = customer_ids.size

In [162]:
order_dates = pd.date_range('2023-01-01', '2024-01-31', periods=N)

In [163]:
order_amounts = 111 + np.random.exponential(1, N) * 10000

In [164]:
platforms = np.random.choice(['mobile', 'desktop', 'shop'], N, p=[0.5, 0.3, 0.2])

In [165]:
categories = np.random.choice(['smartphones', 'laptops', 'watches'], N, p=[0.5, 0.3, 0.2])

In [166]:
df = pd.DataFrame({
    'order_id': range(1, N + 1), 
    'customer_id': customer_ids, 
    'order_date': order_dates, 
    'order_amount': order_amounts, 
    'platform': platforms, 
    'category': categories
})

In [167]:
df.dtypes

order_id                 int64
customer_id              int32
order_date      datetime64[ns]
order_amount           float64
platform                object
category                object
dtype: object

In [168]:
df.order_id = df.order_id.astype('int32')

In [169]:
df.platform = df.platform.astype('category')

In [170]:
df.category = df.category.astype('category')

In [171]:
df.dtypes

order_id                 int32
customer_id              int32
order_date      datetime64[ns]
order_amount           float64
platform              category
category              category
dtype: object

In [98]:
# df.to_pickle('data.pkl')

In [198]:
# времядату превращаем в дату
df['order_day'] = df.order_date.dt.to_period('D')

In [173]:
# времядату превращаем в месяц
df['order_month'] = df.order_date.dt.to_period('M')

In [174]:
# сколько дней прошло с первой покупки
df['day_from_start'] = df['order_day'].astype(int) - df.groupby('customer_id')['order_day'].transform('min').astype(int)
df['day_from_start'] = df['day_from_start'].astype('int32')

In [175]:
# для заказов сделанных в течение 10 дней посчитаем сколько заказов сделано за эти дни
df['purchase_in_10_day'] = df[df.day_from_start < 10].groupby('customer_id').order_id.transform('count')

In [187]:
suitable_orders.sample(5)

,order_id,customer_id,order_date,order_amount,platform,category,order_day,order_month,day_from_start,purchase_in_10_day
288,289,18,2023-01-13 11:37:52.519754170,6093.162687,desktop,smartphones,2023-01-13,2023-01,5,9.0
3767,3768,86,2023-06-13 07:08:06.742756804,7540.898157,mobile,smartphones,2023-06-13,2023-06,0,2.0
48,49,3,2023-01-03 01:56:18.753292361,4209.369596,mobile,laptops,2023-01-03,2023-01,1,30.0
36,37,3,2023-01-02 13:27:14.064969271,16834.002975,desktop,watches,2023-01-02,2023-01,0,30.0
142,143,15,2023-01-07 03:44:05.478489903,1794.915328,mobile,laptops,2023-01-07,2023-01,1,12.0


In [199]:
df.sample(20)

,order_id,customer_id,order_date,order_amount,platform,category,order_day,order_month,day_from_start,purchase_in_10_day
4877,4878,13,2023-07-31 09:57:50.412642668,28605.573084,desktop,laptops,2023-07-31,2023-07,204,NaN
165,166,10,2023-01-08 03:39:49.464442493,10567.156952,shop,smartphones,2023-01-08,2023-01,7,25.0
3799,3800,71,2023-06-14 16:25:39.244951712,341.697944,mobile,laptops,2023-06-14,2023-06,23,NaN
6062,6063,20,2023-09-20 18:49:18.384547848,37202.160758,mobile,smartphones,2023-09-20,2023-09,258,NaN
5190,5191,13,2023-08-13 23:36:17.699736612,3089.843973,desktop,smartphones,2023-08-13,2023-08,217,NaN
162,163,10,2023-01-08 00:32:33.292361720,20408.323928,shop,laptops,2023-01-08,2023-01,7,25.0
237,238,10,2023-01-11 06:34:17.594381036,22703.230395,mobile,smartphones,2023-01-11,2023-01,10,NaN
7397,7398,66,2023-11-17 15:44:14.960491660,6854.276727,desktop,smartphones,2023-11-17,2023-11,229,NaN
8469,8470,88,2024-01-03 03:01:53.784021072,5527.099486,mobile,smartphones,2024-01-03,2024-01,177,NaN
409,410,17,2023-01-18 17:31:04.793678665,6837.483751,shop,watches,2023-01-18,2023-01,12,NaN
